In [51]:
import tqdm
import json
from vowpalwabbit import pyvw

anchor_json = "wiki/nlwiki-20220301/experiments/clean-utrecht.json"

# modelfile = "wiki/nlwiki-20220301/experiments/clean-utrecht.bal.ldf.24b.ldf.vw"
# modelfile = "wiki/nlwiki-20220301/experiments/clean-utrecht.bal.ldf.32b.ldf.vw"
# datafile = "wiki/nlwiki-20220301/experiments/clean-utrecht.bal.ldf.dat"

# modelfile = "wiki/nlwiki-20220301/experiments/clean-utrecht.bal.ldf.ns.24b.ldf.vw"
# datafile = "wiki/nlwiki-20220301/experiments/clean-utrecht.bal.ldf.ns.dat"

modelfile = "wiki/nlwiki-20220301/experiments/clean-utrecht.bal.ldf.ns.reallybal.24b.ldf.vw"
datafile = "wiki/nlwiki-20220301/experiments/clean-utrecht.bal.ldf.ns.dat.reallybal"

# modelfile = "wiki/nlwiki-20220301/experiments/clean-utrecht.ldf.32b.ldf.vw"
# datafile = "wiki/nlwiki-20220301/experiments/clean-utrecht.ldf.dat"

# modelfile = "wiki/nlwiki-20220301/experiments/clean-utrecht.ldf.16b.ldf.vw"
# datafile = "wiki/nlwiki-20220301/experiments/clean-utrecht.ldf.dat.old"

# modelfile = "wiki/nlwiki-20220301/experiments/clean-utrecht.ldf.ns.24b.ldf.vw"
# datafile = "wiki/nlwiki-20220301/experiments/clean-utrecht.ldf.ns.dat"


surface_weights = json.load(open(anchor_json))
surface_weights = {
    m:{int(e.replace('Q','')):c for e,c in ec.items()} 
    for m, ec in surface_weights.items()
}
ents = set(e for ec in surface_weights.values() for e in ec)
e_i = {e:i for i,e in enumerate(sorted(ents))}
i_e = dict(enumerate(sorted(ents)))

print(len(e_i))

# audit = 'wiki/nlwiki-20220301/experiments/clean-q0.25.hash262144.utrecht.vw.audit'
# !rm $audit
model = pyvw.Workspace(
    initial_regressor=modelfile,
    # audit_regressor='/dev/null',
)

28


creating quadratic features for pairs: ls
using no cache
Reading datafile = none
num sources = 0
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
Enabled reductions: gd, scorer-identity, csoaa_ldf, shared_feature_merger
Input label = cs
Output pred = multiclass
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features

finished run
number of examples = 0
weighted example sum = 0.000000
weighted label sum = 0.000000
average loss = n.a.
total feature number = 0


In [52]:
def read_instances(datafile):
    instance = []
    for line in open(datafile):
        line = line.strip()
        if not line:
            if len([l for l in instance if ':0' in l]) == 1:
                instance = [instance[0]] + sorted(instance[1:])[::-1]
                yield instance
            instance = []
        else:
            instance.append(line)
    if len([l for l in instance if ':0' in l]) == 1:
        yield instance

for ex in read_instances(datafile):
    print(*ex, sep='\n')
    print(model.predict(ex))
    break

shared |s vanwege de voortgaande secularisatie worden ook verdere reorganisaties verwacht of zijn deze bezig in de andere noordelijke bisdommen volgens plan zal het aantal parochies in het aartsbisdom en in de bisdommen haarlem rotterdam en breda afnemen met 85 606 parochies
9:0 |l 9
8:5 |l 8
6:2 |l 6
5:2 |l 5
3:9 |l 3
2:8 |l 2
27:2 |l 27
25:4 |l 25
21:2 |l 21
18:3 |l 18
15:2 |l 15
13:6 |l 13
12:5 |l 12
0


0.000000 0.000000            1            1.0          known              0     1066


In [53]:
# Resample to create actually balanced dataset

# inst = [tuple(i) for i in read_instances(datafile)]
# print(len(inst))

# df = pd.DataFrame({'i':inst, 'y':y_true}).merge((pd.Series(y_true).value_counts() > 100).rename('ok'), left_on='y', right_index=True)
# df = df[df['ok']].drop_duplicates().groupby('y').head(100).sample(frac=1)
# with open(datafile + '.reallybal', 'w') as fw:
#     for ex in list(df['i']):
#         for line in ex:
#             print(line, file=fw)
#         print(file=fw)

In [54]:
from collections import *
y_true = [int(x) for es in read_instances(datafile) for e in es if ':0' in e for x, y in [e.split(':',1)] if y.startswith('0')]
y_pred = [model.predict(ex) for ex in read_instances(datafile)]
len(y_true), len(y_pred)

1.000000 2.000000            2            2.0          known              0     1066
1.000000 1.000000            4            4.0          known              0      364
1.000000 1.000000            8            8.0          known              0      988
1.250000 1.500000           16           16.0          known              0     1664
1.062500 0.875000           32           32.0          known              0     1380
1.093750 1.125000           64           64.0          known              0     1920
1.101562 1.109375          128          128.0          known              0     1770
1.050781 1.000000          256          256.0          known              0     1740
1.212891 1.375000          512          512.0          known              0     1066
1.204102 1.195312         1024         1024.0          known              0      960


(1300, 1300)

In [55]:
select_ents = ents

import pandas as pd
import sqlite3
from collections import *

con = sqlite3.connect('wiki/nlwiki-20220301/index_nlwiki-20220301.db')
ent_label = {}
for e in select_ents:
    l = pd.read_sql_query(f'select * from mapping where wikidata_id="Q{e}" limit 1', con)
    ent_label[e] = l['wikipedia_title'][0]
    
dict(Counter(ent_label[e] for i in y_true for e in [i_e[i]] if e in ent_label).most_common())

{'Aartsbisdom_Utrecht_(rooms-katholiek)': 100,
 'Heerlijkheid_Utrecht': 100,
 'Station_Utrecht_Centraal': 100,
 'Groningen_(provincie)': 100,
 'GHHC_Groningen': 100,
 'Utrecht_(stad)': 100,
 'Utrecht_(provincie)': 100,
 'Universiteit_Utrecht': 100,
 'Rijksuniversiteit_Groningen': 100,
 'Groningen_(gemeente)': 100,
 'Groningen_(stad)': 100,
 'Sticht_Utrecht': 100,
 'Station_Groningen': 100}

In [57]:
from sklearn.metrics import classification_report
print('true', {ent_label.get(i_e[i]):c for i,c in Counter(y_true).most_common(5)})
print('pred', {ent_label.get(i_e[i]):c for i,c in Counter(y_pred).most_common(5)})
labels, target_names = zip(*{e_i[e]:l for e,l in ent_label.items()}.items())
print(classification_report(y_true, y_pred, labels=labels, target_names=target_names))

true {'Aartsbisdom_Utrecht_(rooms-katholiek)': 100, 'Heerlijkheid_Utrecht': 100, 'Station_Utrecht_Centraal': 100, 'Groningen_(provincie)': 100, 'GHHC_Groningen': 100}
pred {'Groningen_(stad)': 1155, 'FC_Groningen': 98, 'Aartsbisdom_Utrecht_(rooms-katholiek)': 47}
                                       precision    recall  f1-score   support

                 Groningen_(Suriname)       0.00      0.00      0.00         0
                  Utrecht_(provincie)       0.00      0.00      0.00       100
                         FC_Groningen       0.00      0.00      0.00         0
                    Station_Groningen       0.00      0.00      0.00       100
              Utrechts_Conservatorium       0.00      0.00      0.00         0
                          BVC_Utrecht       0.00      0.00      0.00         0
                       Utrecht_(stad)       0.00      0.00      0.00       100
             Station_Utrecht_Centraal       0.00      0.00      0.00       100
          Rijksuniversit

/home/bbkruit/scratch/anaconda3/envs/takco2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bbkruit/scratch/anaconda3/envs/takco2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bbkruit/scratch/anaconda3/envs/takco2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

In [83]:
fname = 'wiki/nlwiki-20220301/experiments/clean-utrecht.bal.ldf.ns.dat.24b.ldf.vw.inverted'
ls = [l.split()[0].split(':') for l in open(fname).readlines() if '*' in l]
df = pd.DataFrame(ls)
ind = df[0].str[2:].str.split('*', expand=True)
i_l = {e_i[e]:l for e, l in ent_label.items()}
ind[0] = ind[0].apply(lambda i: i_l[int(i)])
df = df[2].rename(0).astype('float').to_frame().set_index(pd.MultiIndex.from_frame(ind))

def topfeat(gr):
    gr = gr.droplevel(0).rename(columns={0:'weight'})
    return gr.loc[gr['weight'].apply('abs').sort_values().index[::-1]].head(10).sort_values('weight').reset_index()

df.groupby(level=0).apply(topfeat).swaplevel().unstack().swaplevel(axis=1).sort_index(axis=1).T

0               1  \
0                                                                              
Aartsbisdom_Utrecht_(rooms-katholiek) 1          s^kardinaal   s^aartsbisdom   
                                      weight       -0.188351       -0.168453   
BVC_Utrecht                           1          s^categorie      s^centraal   
                                      weight       -0.168279        -0.13896   
Bisdom_Groningen-Leeuwarden           1        s^hoofdklasse         s^delft   
                                      weight         -0.1185       -0.086516   
Donar_(basketbalclub)                 1                 s^cs     s^categorie   
                                      weight       -0.111519       -0.100125   
FC_Groningen                          1          s^categorie        s^hockey   
                                      weight        -0.21447       -0.146946   
FC_Utrecht                            1          s^categorie    s^stadhouder   
                                      weight       -0.199696       -0.114053   
GHHC_Groningen                        1            s^seizoen          s^play   
                                      weight       -0.323501       -0.266533   
Groningen_(Suriname)                  1             s^vorige          s^ligt   
                                      weight       -0.083746        0.080047   
Groningen_(gemeente)                  1               s^dorp       s^veendam   
                                      weight       -0.172582        0.167295   
Groningen_(provincie)                 1               s^play  s^universiteit   
                                      weight        0.202262        0.206559   
Groningen_(stad)                      1            s^station  s^degradeerden   
                                      weight        0.217929        0.218278   
Heerlijkheid_Groningen                1                 s^cs   s^roodeschool   
                                      weight       -0.115078        0.080857   
Heerlijkheid_Utrecht                  1         s^stadhouder       s^zeeland   
                                      weight       -0.246839       -0.232625   
Hr.Ms._Utrecht_(1901)                 1          s^categorie      s^centraal   
                                      weight       -0.158313       -0.127476   
Kardinge_(ijsbaan)                    1                 s^cs     s^categorie   
                                      weight       -0.099297       -0.096463   
Ommelanden_(Groningen)                1         s^voorgaande       s^seizoen   
                                      weight       -0.151842       -0.147627   
Rechtbank_Groningen                   1          s^categorie       s^seizoen   
                                      weight       -0.164724       -0.162499   
Rijksuniversiteit_Groningen           1                s^hij    s^hoogleraar   
                                      weight       -0.267025       -0.202003   
Stad_en_Lande                         1        s^hoofdklasse      s^sneeuwen   
                                      weight       -0.097055       -0.095873   
Station_Groningen                     1         s^leeuwarden       s^station   
                                      weight       -0.247634        -0.22797   
Station_Utrecht_Centraal              1            s^station     s^spoorlijn   
                                      weight       -0.262581       -0.220063   
Sticht_Utrecht                        1            s^edelman      s^bisschop   
                                      weight       -0.160412       -0.146449   
Universiteit_Utrecht                  1       s^universiteit    s^hoogleraar   
                                      weight       -0.291374       -0.270618   
Utrecht_(Zuid-Afrika)                 1          s^categorie    s^stadhouder   
                                      weight       -0.197567       -0.118188   
Utrecht_(provincie)                   1          s^provinci